# Encrypted Computations Demo using Google Colab

## Installation
### 1. Download the SEAL-Python library

In [2]:
!git clone https://github.com/Huelse/SEAL-Python

fatal: destination path 'SEAL-Python' already exists and is not an empty directory.


### 2. Buld SEAL C++ code

In [3]:
!cd SEAL-Python/SEAL/native/src;cmake .;make

-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Build type (CMAKE_BUILD_TYPE): Release
-- Microsoft SEAL debug mode: OFF
-- Library build type (SEAL_LIB_BUILD_TYPE): Static_PIC
-- Looking for C++ include x86intrin.h
-- Looking for C++ include x86intrin.h - found
-- Could NOT find MSGSL (missing: MSGSL_INCLUDE_DIR) 
-- Found ZLIB: /usr/lib/x86_64-linux-gnu/libz.so (found suitable exact version "1.2.11") 
-- Looking for pthread.h
-- Looking for pthr

### 3. Download dependencies

In [4]:
!pip3 install -r SEAL-Python/requirements.txt

     |████████████████████████████████| 296kB 2.7MB/s 


### 4. Build SEAL-Python

In [5]:
!cd SEAL-Python;python3 setup.py build_ext -i

running build_ext
building 'seal' extension
creating build
creating build/temp.linux-x86_64-3.6
creating build/temp.linux-x86_64-3.6/src
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I./pybind11/include -I./SEAL/native/src -I/usr/include/python3.6m -c src/base64.cpp -o build/temp.linux-x86_64-3.6/src/base64.o -std=c++17
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I./pybind11/include -I./SEAL/native/src -I/usr/include/python3.6m -c src/wrapper.cpp -o build/temp.linux-x86_64-3.6/src/wrapper.o -std=c++17
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wd

### Install SEAL-Python

In [7]:
!cd SEAL-Python;python3 setup.py install

running install
running build
running build_ext
building 'seal' extension
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I./pybind11/include -I./SEAL/native/src -I/usr/include/python3.6m -c src/base64.cpp -o build/temp.linux-x86_64-3.6/src/base64.o -std=c++17
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.6m -I./pybind11/include -I./SEAL/native/src -I/usr/include/python3.6m -c src/wrapper.cpp -o build/temp.linux-x86_64-3.6/src/wrapper.o -std=c++17
creating build/lib.linux-x86_64-3.6
x86_64-linux-gnu-g++ -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=

### Create a key pair, encryptor and decryptoor

In [13]:
from seal import *

parms = EncryptionParameters(scheme_type.BFV)

poly_modulus_degree = 4096
parms.set_poly_modulus_degree(poly_modulus_degree)
parms.set_coeff_modulus(CoeffModulus.BFVDefault(poly_modulus_degree))
parms.set_plain_modulus(256)
context = SEALContext.Create(parms)

keygen = KeyGenerator(context)
public_key = keygen.public_key()
secret_key = keygen.secret_key()

encryptor = Encryptor(context, public_key)
evaluator = Evaluator(context)
decryptor = Decryptor(context, secret_key)

### Encrypt two hex values

In [27]:
enc1 = Ciphertext()
enc2 = Ciphertext()

encryptor.encrypt(Plaintext("5"), enc1)
encryptor.encrypt(Plaintext("7"), enc2)


Add two encypted numbers

In [29]:
evaluator.add_inplace(enc1, enc2)

In [32]:
# decrypt
result = Plaintext()
decryptor.decrypt(enc1, result)
print(f"result: 0x{result.to_string()}")


result: 0xC


Miltiply and decrypt

In [33]:
evaluator.multiply_inplace(enc1, enc2)
result_mul = Plaintext()
decryptor.decrypt(enc1, result_mul)
print(f"result: 0x{result_mul.to_string()}")

result: 0x54


### Test from
https://github.com/Huelse/SEAL-Python/blob/master/tests/1_bfv_basics.py

In [34]:
print("-" * 50)
x = "6"
x_plain = Plaintext(x)
print("Express x = " + x + " as a plaintext polynomial 0x" +
      x_plain.to_string() + ".")

print("-" * 50)
x_encrypted = Ciphertext()
print("Encrypt x_plain to x_encrypted.")
encryptor.encrypt(x_plain, x_encrypted)

print("    + size of freshly encrypted x: " + str(x_encrypted.size()))

print("    + noise budget in freshly encrypted x: " +
      str(decryptor.invariant_noise_budget(x_encrypted)) + " bits")

x_decrypted = Plaintext()
print("    + decryption of x_encrypted: ", end="")
decryptor.decrypt(x_encrypted, x_decrypted)
print("0x" + x_decrypted.to_string() + " ...... Correct.")

print("-"*50)
print("Compute x_sq_plus_one (x^2+1).")

x_sq_plus_one = Ciphertext()
evaluator.square(x_encrypted, x_sq_plus_one)
plain_one = Plaintext("1")
evaluator.add_plain_inplace(x_sq_plus_one, plain_one)

print("    + size of x_sq_plus_one: " + str(x_sq_plus_one.size()))
print("    + noise budget in x_sq_plus_one: " +
      str(decryptor.invariant_noise_budget(x_sq_plus_one)) + " bits")

decrypted_result = Plaintext()
print("    + decryption of x_sq_plus_one: ", end="")
decryptor.decrypt(x_sq_plus_one, decrypted_result)
print("0x" + decrypted_result.to_string() + " ...... Correct.")

"""
Next, we compute (x + 1)^2.
"""
print("-"*50)
print("Compute x_plus_one_sq ((x+1)^2).")
x_plus_one_sq = Ciphertext()
evaluator.add_plain(x_encrypted, plain_one, x_plus_one_sq)
evaluator.square_inplace(x_plus_one_sq)
print("    + size of x_plus_one_sq: " + str(x_plus_one_sq.size()))
print("    + noise budget in x_plus_one_sq: " +
      str(decryptor.invariant_noise_budget(x_plus_one_sq)) + " bits")
decryptor.decrypt(x_plus_one_sq, decrypted_result)
print("    + decryption of x_plus_one_sq: 0x" +
      decrypted_result.to_string() + " ...... Correct.")

"""
Finally, we multiply (x^2 + 1) * (x + 1)^2 *4.
"""
print("-"*50)
print("Compute encrypted_result (4(x^2+1)(x+1)^2).")
encrypted_result = Ciphertext()
plain_four = Plaintext("4")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
evaluator.multiply(x_sq_plus_one, x_plus_one_sq, encrypted_result)
print("    + size of encrypted_result: " + str(encrypted_result.size()))
print("    + noise budget in encrypted_result: " +
      str(decryptor.invariant_noise_budget(encrypted_result)) + " bits")
print("NOTE: Decryption can be incorrect if noise budget is zero.")
print("\n~~~~~~ A better way to calculate 4(x^2+1)(x+1)^2. ~~~~~~")

print("-"*50)
print("Generate relinearization keys.")
relin_keys = keygen.relin_keys()

"""
We now repeat the computation relinearizing after each multiplication.
"""
print("-"*50)
print("Compute and relinearize x_squared (x^2),")
print(" "*13 + "then compute x_sq_plus_one (x^2+1)")
x_squared = Ciphertext()
evaluator.square(x_encrypted, x_squared)
print("    + size of x_squared: " + str(x_squared.size()))
evaluator.relinearize_inplace(x_squared, relin_keys)
print("    + size of x_squared (after relinearization): " + str(x_squared.size()))
evaluator.add_plain(x_squared, plain_one, x_sq_plus_one)
print("    + noise budget in x_sq_plus_one: " +
      str(decryptor.invariant_noise_budget(x_sq_plus_one)) + " bits")
decryptor.decrypt(x_sq_plus_one, decrypted_result)
print("    + decryption of x_sq_plus_one: 0x" +
      decrypted_result.to_string() + " ...... Correct.")

print("-"*50)
x_plus_one = Ciphertext()
print("Compute x_plus_one (x+1),")
print(" "*13 + "then compute and relinearize x_plus_one_sq ((x+1)^2).")
evaluator.add_plain(x_encrypted, plain_one, x_plus_one)
evaluator.square(x_plus_one, x_plus_one_sq)
print("    + size of x_plus_one_sq: " + str(x_plus_one_sq.size()))
evaluator.relinearize_inplace(x_plus_one_sq, relin_keys)
print("    + noise budget in x_plus_one_sq: " +
      str(decryptor.invariant_noise_budget(x_plus_one_sq)) + " bits")
decryptor.decrypt(x_plus_one_sq, decrypted_result)
print("    + decryption of x_plus_one_sq: 0x" +
      decrypted_result.to_string() + " ...... Correct.")

print("-"*50)
print("Compute and relinearize encrypted_result (4(x^2+1)(x+1)^2).")
evaluator.multiply_plain_inplace(x_sq_plus_one, plain_four)
evaluator.multiply(x_sq_plus_one, x_plus_one_sq, encrypted_result)
print("    + size of encrypted_result: " + str(encrypted_result.size()))
evaluator.relinearize_inplace(encrypted_result, relin_keys)
print("    + size of encrypted_result (after relinearization): " +
      str(encrypted_result.size()))
print("    + noise budget in encrypted_result: " +
      str(decryptor.invariant_noise_budget(encrypted_result)) + " bits")
print("\nNOTE: Notice the increase in remaining noise budget.")

print("-"*50)
print("Decrypt encrypted_result (4(x^2+1)(x+1)^2).")
decryptor.decrypt(encrypted_result, decrypted_result)
print("    + decryption of 4(x^2+1)(x+1)^2 = 0x" +
      decrypted_result.to_string() + " ...... Correct.")

"""
For x=6, 4(x^2+1)(x+1)^2 = 7252. Since the plaintext modulus is set to 1024,
this result is computed in integers modulo 1024. Therefore the expected output
should be 7252 % 1024 == 84, or 0x54 in hexadecimal.
"""


--------------------------------------------------
Express x = 6 as a plaintext polynomial 0x6.
--------------------------------------------------
Encrypt x_plain to x_encrypted.
    + size of freshly encrypted x: 2
    + noise budget in freshly encrypted x: 57 bits
    + decryption of x_encrypted: 0x6 ...... Correct.
--------------------------------------------------
Compute x_sq_plus_one (x^2+1).
    + size of x_sq_plus_one: 3
    + noise budget in x_sq_plus_one: 37 bits
    + decryption of x_sq_plus_one: 0x25 ...... Correct.
--------------------------------------------------
Compute x_plus_one_sq ((x+1)^2).
    + size of x_plus_one_sq: 3
    + noise budget in x_plus_one_sq: 37 bits
    + decryption of x_plus_one_sq: 0x31 ...... Correct.
--------------------------------------------------
Compute encrypted_result (4(x^2+1)(x+1)^2).
    + size of encrypted_result: 5
    + noise budget in encrypted_result: 8 bits
NOTE: Decryption can be incorrect if noise budget is zero.

~~~~~~ A bette

'\nFor x=6, 4(x^2+1)(x+1)^2 = 7252. Since the plaintext modulus is set to 1024,\nthis result is computed in integers modulo 1024. Therefore the expected output\nshould be 7252 % 1024 == 84, or 0x54 in hexadecimal.\n'